<h1 style="color: green; text-align: center; font-size:1.5em;">Продвинутый SQL (Вложенные запросы, объеденение таблиц)</h1>

In [4]:
import sqlite3 as sql
from sqlite3 import Error

In [18]:
conn = sql.connect("BookShop.db")
print("Database created")  if conn else print("Database not created")

Database created


In [19]:
cur = conn.cursor()

CREATE TABLE

In [54]:
createBook = """Create table if not exists Books(id_book INTEGER PRIMARY KEY autoincrement, book_title TEXT, 
pages INTEGER, id_author INTEGER, id_genre INTEGER, 
FOREIGN KEY(id_author) REFERENCES Authors(id_author) ON UPDATE SET NULL ON DELETE SET NULL);"""
try:
    cur.execute(createBook)
    print("Table Book was created")
except Error as e:
    print(f"Error: {e}")


Table Book was created


In [55]:
createAuthor = """Create table if not exists Authors(id_author INTEGER PRIMARY KEY autoincrement, name_author TEXT, 
date_birthday TEXT, gender TEXT);"""
try:
    cur.execute(createAuthor)
    print("Table Author was created")
except Error as e:
    print(f"Error: {e}")

Table Author was created


<h2>INSERT DATA TO TABLE</h2>

In [21]:
books = [(None, "Война и мир", 1245, 1, 2), (None, "Harry Potter", 356, 2, 3), (None, "Jane Eyre", 320, 3, 1)]
try:
    cur.executemany("Insert into Books values(?, ?, ?, ?, ?)", books)
    print("Insert data in Books")
except Error as e:
    print(f"Error: {e}")

Insert data in Books


<h2> ПОДЗАПРОС (SUBQUERY) </h2>

<img src="images/subquery.jpg">

In [29]:
select = "Select * from Books where id_author =  (Select id_author from Authors where name_author = 'Joane Rolling');"
try:
    selectByAuthor = cur.execute(select)
    res  = selectByAuthor.fetchall()
    for row in res:
        print(row)

except Error as e:
    print(f"Error: {e}")

(2, 'Harry Potter', 356, 2, 3)
(4, 'Harry Potter and the prisoner of Azkaban', 420, 2, 3)
(6, 'Harry Potter', 356, 2, 3)


<img src="images/union.png">

In [26]:
select = "Select  id_book, book_title from Books UNION SELECT id_author, name_author  FROM Authors;"
try:
    selectByAuthor = cur.execute(select)
    res  = selectByAuthor.fetchall()
    for row in res:
        print(row)

except Error as e:
    print(f"Error: {e}")

(1, 'L.N.Tolstoy')
(1, 'Война и мир')
(2, 'Harry Potter')
(2, 'Joane Rolling')
(3, 'Jane Eyre')
(3, 'Sharlotte Bronte')
(4, 'Arthur Connan Doel')
(4, 'Harry Potter and the prisoner of Azkaban')
(5, 'Война и мир')
(6, 'Harry Potter')
(7, 'Jane Eyre')


<h2>СОЕДИНЕНИЕ</h2>


In [31]:
select = "Select * from Books as b, Authors as a where a.id_author = b.id_author and date_birthday Like '18%';"
try:
    selectByAuthor = cur.execute(select)
    res  = selectByAuthor.fetchall()
    for row in res:
        print(row)

except Error as e:
    print(f"Error: {e}")

(1, 'Война и мир', 1245, 1, 2, 1, 'L.N.Tolstoy', '1820-02-12', 'male')
(3, 'Jane Eyre', 320, 3, 1, 3, 'Sharlotte Bronte', '1895-01-25', 'female')
(5, 'Война и мир', 1245, 1, 2, 1, 'L.N.Tolstoy', '1820-02-12', 'male')
(7, 'Jane Eyre', 320, 3, 1, 3, 'Sharlotte Bronte', '1895-01-25', 'female')


<h2>INNER JOIN (СОЕДИНЕНИЕ)</h2>

<img src="images/innerJoin.gif">

In [30]:
select = """Select a.name_author, COUNT(b.id_book) from Authors as a INNER JOIN 
Books as b ON a.id_author = b.id_author group by a.name_author;"""
try:
    selectByAuthor = cur.execute(select)
    res  = selectByAuthor.fetchall()
    for row in res:
        print(row)

except Error as e:
    print(f"Error: {e}")

('Joane Rolling', 3)
('L.N.Tolstoy', 2)
('Sharlotte Bronte', 2)


<h1> LEFT JOIN</h1>

<img src="images/leftjoin.gif">

In [32]:
select = "Select name_author, book_title, pages from Authors as a LEFT JOIN Books as b ON a.id_author = b.id_author;"
try:
    selectByAuthor = cur.execute(select)
    res  = selectByAuthor.fetchall()
    for row in res:
        print(row)

except Error as e:
    print(f"Error: {e}")

('L.N.Tolstoy', 'Война и мир', 1245)
('L.N.Tolstoy', 'Война и мир', 1245)
('Joane Rolling', 'Harry Potter', 356)
('Joane Rolling', 'Harry Potter', 356)
('Joane Rolling', 'Harry Potter and the prisoner of Azkaban', 420)
('Sharlotte Bronte', 'Jane Eyre', 320)
('Sharlotte Bronte', 'Jane Eyre', 320)
('Arthur Connan Doel', None, None)


In [34]:
select = "Select a.name_author, b.book_title, b.pages from Authors as a CROSS JOIN Books as b ON a.id_author = b.id_author;"
try:
    selectByAuthor = cur.execute(select)
    res  = selectByAuthor.fetchall()
    for row in res:
        print(row)

except Error as e:
    print(f"Error: {e}")

('L.N.Tolstoy', 'Война и мир', 1245)
('L.N.Tolstoy', 'Война и мир', 1245)
('Joane Rolling', 'Harry Potter', 356)
('Joane Rolling', 'Harry Potter', 356)
('Joane Rolling', 'Harry Potter and the prisoner of Azkaban', 420)
('Sharlotte Bronte', 'Jane Eyre', 320)
('Sharlotte Bronte', 'Jane Eyre', 320)


<p>Предложение <code>HAVING</code> позволяет указать условия, которые фильтруют результаты групп в итоговых результатах. Предложение <code>WHERE</code> устанавливает условия для выбранных столбцов, тогда как предложение <code>HAVING</code> устанавливает условия для групп, созданных предложением <code>GROUP BY</code>.</p>

<p>Ниже приведена позиция предложения <code>HAVING</code> в запросе <code>SELECT</code>.</p>

In [15]:
conn.commit()

In [50]:
conn = sql.connect("chinook.db")
print("Database created")  if conn else print("Database not created")

Database created


In [52]:
cur.execute("PRAGMA foreign_keys = ON;")

In [16]:
conn.close()